In [13]:
ns = {
    #"test": 0,
}
widgets = {"test": {"value": 5, "min": 0, "max": 20}}

In [20]:
from ipywidgets import IntSlider
from IPython.display import display as _display

def changed_values(old_values, values):
    """
    :return: dict containing any values that have been added or changed.
    """
    changed = {}
    for k, v in dict(values).items():
        if k in old_values and v == old_values[k]:
            continue
        values.pop(k)
        old_values[k] = v
        changed[k] = v
    return changed

class IntegerVariable:
    def __init__(self, value, min, max):
        self.value = value
        self.min = min
        self.max = max
        
    def __repr__(self):
        return f"<IntegerVariable {self.value}, min={self.min}, max={self.max}>"
        
class IPyWidgetsView:
    """
    View to interact with Shoebot variables as IPython widgets.
    """
    def __init__(self):
        self.widgets = {}
        
    def add_variable(self, name, variable):
        widget = IntSlider(value=variable.value, description=name.capitalize())
        self.widgets[name] = widget
    
    def get_values(self):
        return {name: widget.value for name, widget in self.widgets.items()}
    
    def update_values(self, values):
        for k, v in values.items():
            widget = self.widgets.get(k)
            if widget:
                widget.value = v
                
    def display(self):
        for widget in self.widgets.values():
            _display(widget)

class VariablesModel:
    def __init__(self, values):
        self.old_values = {}
        self.values = values
        
    def add_variable(self, name, variable):
        self.change_value(name, variable.value)
        return variable

    def get_changed_values(self):
        """
        :return: dict containing any values that have been added or changed.
        """
        return changed_values(self.old_values, self.values)

    def change_value(self, name, value):
        """
        Set value and mark as changed.
        """
        self.old_values.pop(name, None)
        self.values[name] = value
            
            
class VariablesController:
    def __init__(self, ns):
        self.model = VariablesModel(ns)
        self.view = IPyWidgetsView()
        
    def add_variables(self, widget_kwargs): #, new_values):
        variables = {}
        for name, kwargs in widget_kwargs.items():
            #if name in new_values:
            #    kwargs["value"] = new_values[name]
            
            variable = IntegerVariable(**kwargs)            
            self.view.add_variable(name, variable)
            variables[name] = variable
            
        return variables

    def update(self):
        """
        Sync up model and view.
        """
        old_values = dict(self.model.old_values) ##
        model_changed = self.model.get_changed_values()
        #view_changed = changed_values(self.model.old_values, self.view.get_values())
        view_changed = changed_values(old_values, self.view.get_values())
        
        merged = dict(model_changed, **view_changed)
        
        print("model_changed", model_changed)
        print("view_changed", view_changed)
        print("merged", merged)
        
        self.view.update_values(model_changed)
        self.model.values.update(merged)
        return merged


class Context:
    def __init__(self):
        if ns is None:
            self.ns = {}
        else:
            self.ns = ns
        self._variables = VariablesController(self.ns)

    def _add_variables(self, widget_kwargs): #, new_values):
        """
        :param new_values:  These values 
        """
        # Called at beginning of bot script to crea
        return self._variables.add_variables(widget_kwargs)  #, new_values)

    def _sync_variables(self):
        """
        Update variable in namespace from widgets
        """
        changed = self._variables.update()
        #self.ns.update(dict(changed))
        return changed


def setup():
    global widgets, ns, ctx
    
    widgets = {"test": {"value": 5, "min": 0, "max": 20}}
    ns = {}
    ctx = Context()
    variables = ctx._add_variables(widgets)

setup()

ctx._sync_variables()
ctx._variables.view.display()

#assert ns is ctx.ns

assert ctx.ns == {"test": 5}

model_changed {}
view_changed {'test': 5}
merged {'test': 5}


IntSlider(value=5, description='Test')

In [21]:
# view change
# Test changing the slider value updates model

setup()
#ctx._sync_variables()
#ctx._sync_variables()  # TODO - two calls does not seem right

print(ctx.ns)
print(ctx._variables.view.widgets)
ctx._variables.view.widgets["test"].value = 1

ctx._sync_variables()
ctx._variables.view.display()


assert ctx.ns["test"] == 1
assert ctx._variables.view.widgets["test"].value == 1

{}
{'test': IntSlider(value=5, description='Test')}
model_changed {}
view_changed {'test': 1}
merged {'test': 1}


IntSlider(value=1, description='Test')

In [25]:
# Test updating model updates slider value
setup()

#ctx._variables.view.widgets["test"].value = 5
ctx.ns["test"] = 10
ctx._sync_variables()
#ctx._sync_variables()

# TODO - *should* view be changed now ?

ctx._variables.view.display()

print(ctx.ns["test"])
print(ctx._variables.view.widgets)

assert ctx.ns["test"] == 10  # TODO
assert ctx._variables.view.widgets["test"].value == 10

model_changed {'test': 10}
view_changed {'test': 5}
merged {'test': 5}


IntSlider(value=10, description='Test')

5
{'test': IntSlider(value=10, description='Test')}


AssertionError: 

In [ ]:
{"a": 1, "b": {"type": "int", "min": 0, "max": 10, "value": 100}}

In [ ]:
# In Shoebot 1.x, you could pass values for variables - but only those that use add_variable in the code.
# For Shoebot 2, we want to set anything in the namespace..

# Option 1:   Pass type info first 
# Option 2:   Run bot frame and get type info...

# Option 3:   Analyse code to find types :)